In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from openpyxl import Workbook
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)


url = "https://www.musinsa.com/products/3143803"
driver.get(url)

try :
    review_tab = WebDriverWait(driver,10).until(
        EC.element_to_be_clickable((By.XPATH,"//span[contains(text(),'스냅·후기')]"))
    )
    review_tab.click()
    time.sleep(2)
except Exception as e :
    print("리뷰 탭 클릭 실패 : ", e)

scroll_pause = 1
last_height = driver.execute_script("return document.body.scrollHeight")
for _ in range(5) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height :
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source, "html.parser")

product_name = soup.select_one("span.text-title_18px_med.sc-1omefes-1.exqQRL.font-pretendard").get_text(strip=True)

review_cards = soup.select("div.GoodsReviewStaticList__Container-sc-s1ucwv-0.lnqPxe")

reviews = []
for card in review_cards :
    text_tags = card.select("span.text-body_13px_reg.text-black.font-pretendard")
    for tag in text_tags :
        text = tag.get_text(strip=True)
        if text and len(text) > 10 :
            reviews.append(text)

reviews = list(set(reviews))

print(f"상품명 : {product_name}")
for index, review in enumerate(reviews,start = 1) :
    clean_review = review.replace("\n"," ")
    print(f"{index}. {clean_review}")

상품명 : 엣지 라인 트윌리 스카프 (화이트 & 핑크)
1. 화이트블라우스에 포인트로 하려고 구매 이쁘네요 슬림하게나온 트윌리보다 넓어서 스카프느낌
2. 예쁜데 너무 생화이트라 코디하기 생각보다 어려워요! 부드럽고 원단은 좋은데 쉽게 오염될 것 같습니다
3. 색상이 화사하고 이쁘네요,살짝 넓이가 좁았으면 좋았을듯해요
